In [1]:
import pandas as pd
import numpy as np
import config
import datetime
import sys
from sqlalchemy import text
print(sys.version)
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=3
year=21
dico={}
year, specialite

(21, 3)

In [3]:
spe = pd.read_sql("select * from specialite", config.connection_string)
spe

,id,label,label_long,psp_spe_snds
0,1,Psychiatre,Psychiatre,33.0
1,2,Anest,Anesthésiste,2.0
2,3,Dermato,Dermatologue,5.0
3,4,Gastro,Gastro-entérologue,8.0
4,5,Gyneco,Gynécologue,7.0
5,6,Opthtalmo,Opthtalmogue,15.0
6,7,Pediatre,Pédiatre,12.0
7,8,Radio,Radiologue,6.0
8,9,Dentiste,Dentiste,36.0
9,10,Generaliste,Généraliste,1.0


In [4]:
def get_ps(year, specialite):
    sql=f"""
select sp.specialite_id, s.label_long as label, ds.annee as year, ps.id as p_id, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join date_source ds on ds.id=tds.date_source_id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
where sp.specialite_id={specialite}
and ds.annee={year}
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()


In [5]:
def get_pa(year, specialite):
    sql=f"""
select sp.specialite_id, ds.annee as year, pa.id as p_id, an.lon, an.lat
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    # print(sql)
    return pd.read_sql(sql, config.connection_string).dropna()

In [6]:
def get_rpps(year, specialite):
    sql=f"""
select personne.id p_id,an.lon,an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
where sp.specialite_id={specialite}
and personne.date_maj >= '20{year}-01-01' and date_effet <= '20{year}-12-31'
"""
    print(f"Quering RPPS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string).dropna()

In [7]:
df2=get_pa(23,3)
df2["p_id"].nunique()

Quering PA for year 23 and specialite 3


2481

In [8]:
nb_unique = len(df2.groupby(["p_id", "lon", "lat"]))
nb_unique

3548

In [9]:
nb = df2["p_id"].nunique()
nb

2481

In [10]:
ameli={1:6214,2:3859,3:2433,4:2021,5:4368,6:4295,7:2704,8:5518,9:36062,10:51488,11:4989,12:7777,13:842,14:115,15:1036,16:1971,17:1196,18:505,19:1449,20:704}
ameli

{1: 6214,
 2: 3859,
 3: 2433,
 4: 2021,
 5: 4368,
 6: 4295,
 7: 2704,
 8: 5518,
 9: 36062,
 10: 51488,
 11: 4989,
 12: 7777,
 13: 842,
 14: 115,
 15: 1036,
 16: 1971,
 17: 1196,
 18: 505,
 19: 1449,
 20: 704}

In [11]:
columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","delta_ps","nb_pa_cabinet","nb_pa","delta_pa","nb_pa_ps_cabinet","nb_pa_ps","delta_pa_ps","nb_pa_outer_ps_cabinet","nb_pa_outer_ps","delta_pa_outer_ps","nb_rpps_cabinet","np_rpps","delta_rpps"]
columns=["specialite","label","year","ameli","nb_ps_cabinet","nb_ps","delta_ps","nb_pa_cabinet","nb_pa","delta_pa","nb_rpps_cabinet","np_rpps","delta_rpps"]
df = pd.DataFrame(columns=columns)
for specialite in range(1,21):
    for yy in range(20,26): 
        ps_df=get_ps(yy,specialite)
        label=ps_df["label"][0] if len(ps_df["label"])>0 else ""
        nb_unique_ps=len(ps_df.groupby(["p_id", "lon", "lat"]))
        nb_ps=ps_df["p_id"].nunique()
        pa_df=get_pa(yy,specialite)
        nb_unique_pa=len(pa_df.groupby(["p_id", "lon", "lat"]))
        nb_pa=pa_df["p_id"].nunique()
        rpps_df=get_rpps(yy,specialite)
        nb_unique_rpps=len(rpps_df.groupby(["p_id", "lon", "lat"]))
        nb_rpps=rpps_df["p_id"].nunique()
        a=np.nan
        if yy==23 and specialite in ameli:
            a=ameli[specialite]
        # df = pd.concat([pd.DataFrame([[specialite,label,yy,a,nb_unique_ps,nb_ps,(nb_ps-a)/a,nb_unique_pa,nb_pa,(nb_pa-a)/a,nb_unique_pa_ps,nb_pa_ps,(nb_pa_ps-a)/a,nb_unique_pa_ops,nb_pa_ops,(nb_pa_ops-a)/a,nb_unique_rpps,nb_rpps,(nb_rpps-a)/a]], columns=df.columns), df], ignore_index=True)
        df = pd.concat([pd.DataFrame([[specialite,label,yy,a,nb_unique_ps,nb_ps,(nb_ps-a)/a,nb_unique_pa,nb_pa,(nb_pa-a)/a,nb_unique_rpps,nb_rpps,(nb_rpps-a)/a]], columns=df.columns), df], ignore_index=True)
df=df.sort_values(by=['specialite',"year"])
df

Quering PS for year 20 and specialite 1
Quering PA for year 20 and specialite 1
Quering RPPS for year 20 and specialite 1
Quering PS for year 21 and specialite 1
Quering PA for year 21 and specialite 1
Quering RPPS for year 21 and specialite 1
Quering PS for year 22 and specialite 1
Quering PA for year 22 and specialite 1
Quering RPPS for year 22 and specialite 1
Quering PS for year 23 and specialite 1
Quering PA for year 23 and specialite 1
Quering RPPS for year 23 and specialite 1
Quering PS for year 24 and specialite 1
Quering PA for year 24 and specialite 1
Quering RPPS for year 24 and specialite 1
Quering PS for year 25 and specialite 1
Quering PA for year 25 and specialite 1
Quering RPPS for year 25 and specialite 1
Quering PS for year 20 and specialite 2
Quering PA for year 20 and specialite 2
Quering RPPS for year 20 and specialite 2
Quering PS for year 21 and specialite 2
Quering PA for year 21 and specialite 2
Quering RPPS for year 21 and specialite 2
Quering PS for year 22 a

,specialite,label,year,ameli,nb_ps_cabinet,nb_ps,delta_ps,nb_pa_cabinet,nb_pa,delta_pa,nb_rpps_cabinet,np_rpps,delta_rpps
119,1,Psychiatre,20,NaN,5915,5647,NaN,8505,6002,NaN,16589,16589,NaN
118,1,Psychiatre,21,NaN,6323,5833,NaN,8521,6033,NaN,16910,16910,NaN
117,1,Psychiatre,22,NaN,6158,5766,NaN,8527,6041,NaN,17166,17166,NaN
116,1,Psychiatre,23,6214,5867,5441,-0.124397,8678,6146,-0.010943,17433,17433,1.805439
115,1,Psychiatre,24,NaN,5519,5142,NaN,0,0,NaN,17640,17640,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,20,Stomatologue,21,NaN,473,421,NaN,616,378,NaN,1147,1147,NaN
3,20,Stomatologue,22,NaN,406,356,NaN,563,339,NaN,1151,1151,NaN
2,20,Stomatologue,23,704,341,301,-0.572443,522,316,-0.551136,1151,1151,0.634943
1,20,Stomatologue,24,NaN,297,262,NaN,0,0,NaN,1154,1154,NaN


In [12]:
df.to_excel("ps_stats.xlsx", index=False)
df.to_sql("ps_stats", config.connection_string, schema="apl", if_exists="replace", index=False)

120